In [1]:
import time
from collections import namedtuple
import numpy as np
import heapq
from random import choice
from itertools import count

DIMENSION = 3
move = namedtuple('Move', ['pos1', 'pos2'])
goal = np.array([i for i in range(1, DIMENSION**2)] + [0]).reshape((DIMENSION, DIMENSION))

Helper function for moves

In [2]:

# Available actions to swap the blank space
def get_actions(puzzle: np.ndarray) -> list[move]:
    x, y = [int(_[0]) for _ in np.where(puzzle == 0)]
    moves = list()
    if x > 0:
        moves.append(move((x, y), (x - 1, y)))
    if x < DIMENSION - 1:
        moves.append(move((x, y), (x + 1, y)))
    if y > 0:
        moves.append(move((x, y), (x, y - 1)))
    if y < DIMENSION - 1:
        moves.append(move((x, y), (x, y + 1)))
    return moves

def perform_move(puzzle: np.ndarray, action: move) -> np.ndarray:
    new_puzzle = puzzle.copy()
    new_puzzle[action.pos1], new_puzzle[action.pos2] = new_puzzle[action.pos2], new_puzzle[action.pos1]
    return new_puzzle

Manhattan and linear conflicts heuristics

In [3]:
# Manhattan distance heuristic
def manhattan(puzzle: np.ndarray) -> int:
    distance = 0
    for i in range(DIMENSION):
        for j in range(DIMENSION):
            if puzzle[i, j] != 0:
                target_row, target_col = (puzzle[i, j] - 1) // DIMENSION, (puzzle[i, j] - 1) % DIMENSION
                distance += abs(i - target_row) + abs(j - target_col)
    return distance

# Linear conflict heuristic 
def linear_conflict(puzzle: np.ndarray) -> int:
    conflict = 0
    for i in range(DIMENSION):
        row = puzzle[i, :]
        goal_row = [(val - 1) // DIMENSION for val in row if val != 0]
        conflict += sum(goal_row[j] == i and any(goal_row[k] < goal_row[j] for k in range(j)) for j in range(len(goal_row)))

        col = puzzle[:, i]
        goal_col = [(val - 1) % DIMENSION for val in col if val != 0]
        conflict += sum(goal_col[j] == i and any(goal_col[k] < goal_col[j] for k in range(j)) for j in range(len(goal_col)))

    return conflict

# combining both Manhattan and Linear Conflict
def combined_heuristic(puzzle: np.ndarray) -> int:
    return manhattan(puzzle) + 2*linear_conflict(puzzle)

A* search algorithm 

In [4]:
# A* Search to solve the puzzle
def solve_puzzle(initial_puzzle: np.ndarray):
    open_list = []
    heapq.heappush(open_list, (0, next(unique_id := count()), initial_puzzle, []))
    closed_set = set()
    explored = 0

    while open_list:
        _, _, current_puzzle, path = heapq.heappop(open_list)
        explored += 1

        if np.array_equal(current_puzzle, goal):
            actions = []
            for i in range(1, len(path)):
                # Determine the move from the previous state to the current state
                prev_state = path[i - 1]
                current_state = path[i]
                x1, y1 = np.where(prev_state == 0)
                x2, y2 = np.where(current_state == 0)
                
                if x2 > x1: actions.append('down')
                elif x2 < x1: actions.append('up')
                elif y2 > y1: actions.append('right')
                elif y2 < y1: actions.append('left')

            return path, explored, actions

        puzzle_tuple = tuple(map(tuple, current_puzzle))
        if puzzle_tuple in closed_set:
            continue
        closed_set.add(puzzle_tuple)

        for action in get_actions(current_puzzle):
            next_puzzle = perform_move(current_puzzle, action)
            puzzle_tuple = tuple(map(tuple, next_puzzle))
            if puzzle_tuple not in closed_set:
                g = len(path) + 1
                h = combined_heuristic(next_puzzle)
                f = g + h
                heapq.heappush(open_list, (f, next(unique_id), next_puzzle, path + [next_puzzle]))

    return None, explored, []





Creating a puzzle

In [5]:
RANDOM_STEPS = 100000
current_puzzle = np.array([i for i in range(1, DIMENSION**2)] + [0]).reshape((DIMENSION, DIMENSION))
for _ in range(RANDOM_STEPS):
    current_puzzle = perform_move(current_puzzle, choice(get_actions(current_puzzle)))


Solving a puzzle

In [6]:
start = time.time()
solution_path, total_cost, actions = solve_puzzle(current_puzzle)
end = time.time()


execution_time = end - start
print("Initial state:")
print(current_puzzle)
print("\nGoal state:")
print(goal)
print(f"\n{total_cost} actions evaluated")
if solution_path:
    print(f"Solution found in {len(actions)} steps.")
    print(f"Actions: {actions}")
else:
    print("No solution found.")


Initial state:
[[5 3 2]
 [6 4 1]
 [0 7 8]]

Goal state:
[[1 2 3]
 [4 5 6]
 [7 8 0]]

2169 actions evaluated
Solution found in 21 steps.
Actions: ['right', 'right', 'up', 'left', 'down', 'right', 'down', 'left', 'left', 'up', 'up', 'right', 'down', 'down', 'right', 'up', 'left', 'left', 'down', 'right', 'right']
